In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score

warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("kaggle_train.csv",parse_dates=['date'])

In [3]:
df.head()

,Unnamed: 0,date,store_nbr,sales,onpromotion,cluster,WorkDay,dcoilwtico,transactions,GROCERY I,...,c_Cayambe,c_Cuenca,c_Ambato,c_Daule,c_Loja,c_Manta,c_Babahoyo,c_Libertad,c_Esmeraldas,store_type
0,0,2013-01-01,25,0,0,1,0,93.14,770,0,...,0,0,0,0,0,0,0,0,0,3
1,1,2013-01-01,25,0,0,1,0,93.14,770,0,...,0,0,0,0,0,0,0,0,0,3
2,2,2013-01-01,25,2,0,1,0,93.14,770,0,...,0,0,0,0,0,0,0,0,0,3
3,3,2013-01-01,25,810,0,1,0,93.14,770,0,...,0,0,0,0,0,0,0,0,0,3
4,4,2013-01-01,25,0,0,1,0,93.14,770,0,...,0,0,0,0,0,0,0,0,0,3


In [4]:
df['day']=df['date'].dt.day
df['month']=df['date'].dt.month
df['year']=df['date'].dt.year

In [5]:
df.drop(['Unnamed: 0','date','onpromotion'],1,inplace=True)

In [6]:
df['other']=0

In [7]:
def rmse(y_true,y_pred):
    return np.sqrt(np.square(np.subtract(y_true,y_pred)).mean())

In [8]:
def rsquared(y_true, y_pred):
    rss = np.sum(np.square((y_true- y_pred)))
    mean = np.mean(y_true)
    sst = np.sum(np.square(y_true-mean))
    r_square = 1 - (rss/sst)
    return r_square

In [9]:
performance=pd.DataFrame({"Model":[],
                         "RMSE Error":[],
                         "R squared value":[]})

In [10]:
def checknegative(predict):
    negative=False
    for i in predict:
        if i<0:
            negative=True
    if negative==True:
        print("negative values are present")
    else:
        print("no negative values are present")

In [11]:
def handlenegative(predict):
    pred=[]
    for i in predict:
        if i>0:
            pred.append(i)
        else:
            pred.append(0)
    print("negative values are handled successfully")
    return pred

In [12]:
def normalize(x):
    return (x-min(x))/(max(x)-min(x))

In [13]:
df.head()

,store_nbr,sales,cluster,WorkDay,dcoilwtico,transactions,GROCERY I,BEVERAGES,PRODUCE,CLEANING,...,c_Loja,c_Manta,c_Babahoyo,c_Libertad,c_Esmeraldas,store_type,day,month,year,other
0,25,0,1,0,93.14,770,0,0,0,0,...,0,0,0,0,0,3,1,1,2013,0
1,25,0,1,0,93.14,770,0,0,0,0,...,0,0,0,0,0,3,1,1,2013,0
2,25,2,1,0,93.14,770,0,0,0,0,...,0,0,0,0,0,3,1,1,2013,0
3,25,810,1,0,93.14,770,0,1,0,0,...,0,0,0,0,0,3,1,1,2013,0
4,25,0,1,0,93.14,770,0,0,0,0,...,0,0,0,0,0,3,1,1,2013,0


In [14]:
X=df.drop("sales",1)

In [15]:
y=df.sales

In [16]:
y_norm=normalize(y)

In [17]:
y

0             0
1             0
2             2
3           810
4             0
           ... 
1968478     438
1968479     154
1968480    2419
1968481     121
1968482      16
Name: sales, Length: 1968483, dtype: int64

In [20]:
from sklearn.naive_bayes import GaussianNB

In [21]:
nb=GaussianNB()

In [22]:
X=df.drop("sales",1)
y=df.sales

In [23]:
X_train=X[:-400000]
X_test=X[-400000:]
y_train=y_norm[:-400000]
y_test=y_norm[-400000:]

#we can also use below code
#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=400000,random_state=0)

In [24]:
print("training size: ",X_train.shape[0],"i.e {}% of total dataset".format(round((X_train.shape[0]/X.shape[0])*100)))

training size:  1568483 i.e 80% of total dataset


In [25]:
print("testing size: ",X_test.shape[0],"i.e {}% of total dataset".format(round((X_test.shape[0]/X.shape[0])*100)))

testing size:  400000 i.e 20% of total dataset


In [26]:
nb.fit(X_train,y_train)

GaussianNB()

In [27]:
y_predict=[]

In [28]:
bsize=4000
for batch in range(int(X_test.shape[0]/bsize)):
    y_predict.append(nb.predict(X_test[batch*bsize:(batch+1)*bsize]))

KeyboardInterrupt: 

In [ ]:
print("root mean squared error for testing dataframe: ",rmse(y_test,y_pred))

In [ ]:
print("r squared value: ",rsquared(y_test,y_pred))

In [ ]:
naivebayes={'Model': 'Naive Bayes Regressor', 'RMSE Error': rmse(y_test,y_pred), 'R squared value': rsquared(y_test,y_pred)}
performance = performance.append(naivebayes, ignore_index = True)

In [ ]:
plt.figure(figsize=(10,6))
c = [i for i in range(1,400001,1)]
plt.plot(c,abs(y_test-y_pred))
plt.ylabel("Error")
plt.xlabel("index")
plt.title("Error Terms")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(abs(y_test-y_pred))
plt.ylabel("Error")
plt.xlabel("index")
plt.title("Error Terms")
plt.show()

In [ ]:
error=pd.DataFrame({"true y":y_test.values,
                   "prediction y":y_pred,
                   "error":abs(y_test-y_pred)})

In [ ]:
new_df=pd.concat([X_test,error], axis=1)

In [ ]:
new_df=new_df[new_df['error']<400]